<a href="https://colab.research.google.com/github/orutega/thermal_analysis/blob/master/Thermal_Node_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#熱節点解析プログラム

##Googleドライブでフォルダを参照するために必要な処理

In [0]:
# Google Drive用　Google Drive上のファイルを使用するので，マウントするための処理
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


##インポート

In [0]:
import numpy as np
import sympy as sp
import pandas as pd
import csv

##熱節点の情報をインポート

In [0]:
import csv

csv_file = open('/content/gdrive/My Drive/Research/LOTUS/ThermalAnalyze/Thermal_Node_Definition.csv', "r", encoding="ms932", errors="", newline="" )
#リスト形式
f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)

<_io.TextIOWrapper name='/content/gdrive/My Drive/Research/LOTUS/ThermalAnalyze/Thermal_Node_Definition.csv' mode='r' encoding='ms932'>


In [0]:
csv_input = pd.read_csv(filepath_or_buffer='/content/gdrive/My Drive/Research/LOTUS/ThermalAnalyze/Thermal_Node_Definition.csv', encoding="ms932", sep=",")
# インプットの項目数（行数 * カラム数）を返却します。
print(csv_input)

   Thermal_Conduction_Conductance_Matrix Unnamed: 1  ... Unnamed: 5 Unnamed: 6
0                                   K_ij      node1  ...      node5      node6
1                                  node1        NaN  ...          0          0
2                                  node2        NaN  ...        NaN        NaN
3                                  node3        NaN  ...        NaN        NaN
4                                  node4        NaN  ...        NaN        NaN
5                                  node5        NaN  ...        NaN        NaN
6                                  node6        NaN  ...        NaN        NaN
7   Thermal_Radiation_Conductance_Matrix        NaN  ...        NaN        NaN
8                                   R_ij      node1  ...      node5      node6
9                                  node1        NaN  ...        NaN        NaN
10                                 node2        NaN  ...        NaN        NaN
11                                 node3        NaN 

In [0]:
# T_iを状態変数ベクトルとして, tを時間とする。Node_const_ijは各Node
def thermal_equilibrium_equation(T_state, t, Node_const_ij):
  